In [ ]:
using HDF5
using PyPlot
include("PostProcess.jl")

In [ ]:
ice_shell_thickness_range = range(5.0,60.0,7)  # Define your ice shell thickness range
wavelength_range = range(5.0,300.0,10)  # Define your wavelength range
amplitude = 20.0  # Define the amplitude value
output_path = "combined_output.hdf5"  # Define the output file path
combine_hdf5_files(ice_shell_thickness_range,wavelength_range,amplitude,output_path)

In [ ]:
fname = "combined_output.hdf5"
# Displaying all of the file's information
fid = h5open(fname, "r")

In [ ]:
# Reading Data from file
modeldata = fid["Combined Model Run"];

# Displaying the contents in Model Run group 
for obj in modeldata
    data = read(obj)
    println(obj)
    display(data)
end

# # Accessing Model Run contents that I want
Wavelength = read(modeldata,"Wavelength");
Ice_shell = read(modeldata, "Ice Shell Thickness");
Time_halfspace = read(modeldata,"Viscous Relaxation Time(Half-Space)");
Time_rel = read(modeldata,"Viscous Relaxation Time(Model)");
Time_thickening = read(modeldata, "Thickening Time");
Time_rel_fitted = read(modeldata, "Fitted Viscous Relaxation Time");

# Close file
close(fid)

In [ ]:
println("Wavelength: ",Wavelength/1e3,"(km)")
println("Ice Shell Thickness: ",Ice_shell/1e3,"(km)")
println("Viscous Relaxation Time (Half-Space): ",Time_halfspace/1e3,"(kyr)")
println("Viscous Relaxation Time (Model): ",Time_rel/1e3,"(kyr)")
println("Thickening Time: ",Time_thickening/1e3,"(kyr)")
println("Fitted Viscous Relaxation Time: ",Time_rel_fitted/1e3,"(kyr)")

In [ ]:
# is in time yrs 
# time/1e6 is Millions of Years
# time/1e3 is Thousands of Years

In [ ]:
figure()
cs = contour(Ice_shell/1e3,Wavelength/1e3,Time_rel./Time_thickening)
clabel(cs,inline=true,fmt="%0.1f                       ",colors="w",fontsize=10)
contourf(cs,cmap=get_cmap("viridis"))
title(L"t'")
colorbar(label=L"t'",location="right")
gca().yaxis.set_label_position("left")
gca().set_ylabel(L"\lambda\,(km)")
gca().set_xlabel(L"h_{ice\,shell}\,(km)",rotation=0)
# gca().yaxis.set_label_coords(-0.23,0.5)
gca().spines["top"].set_visible(false) # Hide the top edge of the axis
gca().spines["right"].set_visible(false) # Hide the right edge of the axis
gca().xaxis.set_ticks_position("bottom")
gca().yaxis.set_ticks_position("left")
gca().spines["left"].set_position(("axes",-0.03)) # Offset the left scale from the axis
gca().spines["bottom"].set_position(("axes",-0.03)) # Offset the bottom scale from the axis
# gca().spines["top"].set_color("none") # Remove the top axis boundary
# gca().spines["right"].set_color("none") # Remove the right axis boundary
# gca().set_xlim([10.0,30.0])
# gca().set_ylim([10.0,100.0])
tight_layout()
savefig("Trel_Ttic.jpeg",dpi=300)
show()

figure()
cs = contour(Ice_shell/1e3,Wavelength/1e3,Time_rel/1e3)
clabel(cs,inline=true,fmt="%0.1f                       ",colors="w",fontsize=10)
contourf(cs,cmap=get_cmap("viridis"))
title(L"t_{relaxation}")
colorbar(label=L"t_{relaxation}(kyr)",location="right")
gca().yaxis.set_label_position("left")
gca().set_ylabel(L"\lambda\,(km)")
gca().set_xlabel(L"h_{ice\,shell}\,(km)",rotation=0)
# gca().yaxis.set_label_coords(-0.23,0.5)
gca().spines["top"].set_visible(false) # Hide the top edge of the axis
gca().spines["right"].set_visible(false) # Hide the right edge of the axis
gca().xaxis.set_ticks_position("bottom")
gca().yaxis.set_ticks_position("left")
gca().spines["left"].set_position(("axes",-0.03)) # Offset the left scale from the axis
gca().spines["bottom"].set_position(("axes",-0.03)) # Offset the bottom scale from the axis
# gca().spines["top"].set_color("none") # Remove the top axis boundary
# gca().spines["right"].set_color("none") # Remove the right axis boundary
# gca().set_xlim([10.0,30.0])
# gca().set_ylim([10.0,100.0])
tight_layout()
# savefig("Trel_Ttic.jpeg",dpi=300)
show()

figure()
cs = contour(Ice_shell/1e3,Wavelength/1e3,Time_thickening/1e6)
clabel(cs,inline=true,fmt="%0.1f                       ",colors="w",fontsize=10)
contourf(cs,cmap=get_cmap("viridis"))
title(L"t_{thickening}")
colorbar(label=L"t_{thickening}(Myr)",location="right")
gca().yaxis.set_label_position("left")
gca().set_ylabel(L"\lambda\,(km)")
gca().set_xlabel(L"h_{ice\,shell}\,(km)",rotation=0)
# gca().yaxis.set_label_coords(-0.23,0.5)
gca().spines["top"].set_visible(false) # Hide the top edge of the axis
gca().spines["right"].set_visible(false) # Hide the right edge of the axis
gca().xaxis.set_ticks_position("bottom")
gca().yaxis.set_ticks_position("left")
gca().spines["left"].set_position(("axes",-0.03)) # Offset the left scale from the axis
gca().spines["bottom"].set_position(("axes",-0.03)) # Offset the bottom scale from the axis
# gca().spines["top"].set_color("none") # Remove the top axis boundary
# gca().spines["right"].set_color("none") # Remove the right axis boundary
# gca().set_xlim([10.0,30.0])
# gca().set_ylim([10.0,100.0])
tight_layout()
# savefig("Trel_Ttic.jpeg",dpi=300)
show()